In [1]:
import nbimporter
import dataDownloader as dd

Importing Jupyter notebook from dataDownloader.ipynb


In [2]:
def getPreviousMatches(matches,x,teamVenue,predictionMatches):
    previousMatches = []
    for i in range(x-1,0,-1):
        if(len(previousMatches) == predictionMatches):
            break
        if(matches[x][teamVenue]==matches[i]['HomeTeam'] or matches[x][teamVenue]==matches[i]['AwayTeam']):
            previousMatches.append(matches[i])
    return previousMatches

In [9]:
import glob
import csv
import random
def generateTestAndTrainData(trainToOverallRatio, predictionMatches):
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    #Read files
    filesPaths = glob.glob("../data/*.csv")
    x_data = []
    y_data = []
    for dataFile in filesPaths: 
        print(f"Reading file {dataFile}")
        with open(dataFile) as csvfile:
            reader = csv.DictReader(csvfile)
            #print(f"reading file {dataFile} with {sum(1 for line in reader)} entries")
            matches = []
            for row in reader:
                matches.append(row)
            for x in range(len(matches)):
                homePreviousMatches = getPreviousMatches(matches, x, 'HomeTeam', predictionMatches)
                awayPreviousMatches = getPreviousMatches(matches, x,'AwayTeam',predictionMatches)
                matchLearningData = []
                if len(homePreviousMatches) == predictionMatches and len(awayPreviousMatches) == predictionMatches:
                    #print(f"{x}: {matches[x]['HomeTeam']} vs {matches[x]['AwayTeam']}")

                    for hm in homePreviousMatches:
                        temp = []
                        if matches[x]['HomeTeam'] == hm['HomeTeam']:
                            temp.append(int(float(hm['FTHG']))-int(float(hm['FTAG'])))
                            temp.append(int(hm['HS']))
                            temp.append(int(hm['AS']))
                            temp.append(int(hm['HST']))
                            temp.append(int(hm['AST']))
                            temp.append(int(hm['HC']))
                            temp.append(int(hm['AC']))
                        else:
                            temp.append(int(float(hm['FTAG']))-int(float(hm['FTHG'])))
                            temp.append(int(hm['AS']))
                            temp.append(int(hm['HS']))
                            temp.append(int(hm['AST']))
                            temp.append(int(hm['HST']))
                            temp.append(int(hm['AC']))
                            temp.append(int(hm['HC']))
                        matchLearningData.append(temp)

                    for am in homePreviousMatches:
                        temp = []
                        if matches[x]['AwayTeam'] == am['HomeTeam']:
                            temp.append(int(float(am['FTHG']))-int(float(am['FTAG'])))
                            temp.append(int(am['HS']))
                            temp.append(int(am['AS']))
                            temp.append(int(am['HST']))
                            temp.append(int(am['AST']))
                            temp.append(int(am['HC']))
                            temp.append(int(am['AC']))
                        else:
                            temp.append(int(float(am['FTAG']))-int(float(am['FTHG'])))
                            temp.append(int(am['AS']))
                            temp.append(int(am['HS']))
                            temp.append(int(am['AST']))
                            temp.append(int(am['HST']))
                            temp.append(int(am['AC']))
                            temp.append(int(am['HC']))
                        matchLearningData.append(temp)
                        

                    x_data.append(matchLearningData)
                    y_data.append(int(float(matches[x]['FTHG']))-int(float(matches[x]['FTAG'])))

    for i in range(len(x_data)):
        if(random.random() < trainToOverallRatio):
            x_train.append(x_data[i])
            y_train.append(y_data[i])
        else:
            x_test.append(x_data[i])
            y_test.append(y_data[i])
    print(f"overall matches count: {len(x_data)}")
    print(f"train matches count: {len(x_train)}   {len(x_train)/len(x_data)}% of overall")
    print(f"test matches count: {len(x_test)}")
    print(x_data[0])
    return x_train, y_train, x_test, y_test

In [10]:
generateTestAndTrainData(0.7,5)

Reading file ../data\E0-1920.csv
overall matches count: 328
train matches count: 239   0.7286585365853658% of overall
test matches count: 89
[[1, 11, 17, 7, 4, 6, 12], [0, 10, 21, 2, 8, 2, 3], [2, 12, 12, 4, 3, 5, 8], [-1, 14, 15, 3, 6, 5, 9], [-3, 11, 10, 3, 4, 7, 2], [1, 11, 17, 7, 4, 6, 12], [0, 21, 10, 8, 2, 3, 2], [2, 12, 12, 4, 3, 5, 8], [1, 15, 14, 6, 3, 9, 5], [-3, 11, 10, 3, 4, 7, 2]]


   [2, 14, 12, 5, 3, 6, 5],
   [9, 25, 6, 15, 3, 7, 2],
   [-1, 13, 19, 4, 3, 4, 9],
   [-1, 2, 18, 1, 8, 6, 4],
   [-5, 3, 13, 0, 5, 0, 9]],
  [[-1, 15, 12, 4, 6, 10, 4],
   [2, 21, 10, 11, 3, 10, 1],
   [0, 7, 10, 2, 4, 3, 1],
   [-1, 12, 12, 3, 2, 6, 4],
   [0, 16, 10, 4, 5, 8, 7],
   [-1, 15, 12, 4, 6, 10, 4],
   [2, 21, 10, 11, 3, 10, 1],
   [0, 10, 7, 4, 2, 1, 3],
   [-1, 12, 12, 3, 2, 6, 4],
   [0, 10, 16, 5, 4, 7, 8]],
  [[-1, 16, 20, 3, 9, 4, 5],
   [1, 12, 15, 6, 4, 4, 10],
   [0, 15, 7, 5, 3, 10, 5],
   [0, 11, 10, 2, 1, 6, 7],
   [-1, 10, 12, 2, 2, 5, 14],
   [-1, 16, 20, 3, 9, 4, 5],
   [-1, 15, 12, 4, 6, 10, 4],
   [0, 15, 7, 5, 3, 10, 5],
   [0, 10, 11, 1, 2, 7, 6],
   [-1, 10, 12, 2, 2, 5, 14]],
  [[0, 14, 17, 4, 5, 4, 4],
   [3, 13, 6, 6, 0, 7, 6],
   [0, 10, 12, 4, 4, 4, 10],
   [1, 8, 9, 2, 3, 7, 12],
   [0, 9, 8, 3, 2, 7, 7],
   [0, 14, 17, 4, 5, 4, 4],
   [-3, 6, 13, 0, 6, 6, 7],
   [0, 10, 12, 4, 4, 4, 10],
   [-1, 9, 8, 3, 2, 12, 7],
   [0, 9, 8, 3, 2, 7, 7]],
  